#### Física de la Radioterapia. Master de Física Biomédica. Universidad Complutense de Madrid
# Evaluación de tratamientos de radioterapia. Radiobiología
## Distribuciones de dosis absorbida
-----
*Este cuaderno está basado en los cuadernos de ejemplo del módulo* [`platipy`](https://pyplati.github.io/platipy/getting_started.html)

Los datos corresponden a una paciente real tratada en el Hospital de Fuenlabrada.

Han sido anonimizados de acuerdo a procedimientos validados.

**Objetivos**:

* Mostrar elementos que aparecen en la planificación de un tratamiento de radioterapia

* Comprender las bases radiobiológicas del fraccionamiento de la dosis en radioterapia

## Fundamentos

Los conceptos básicos sobre los que se trabajará en este cuaderno corresponden a un tratamiento de mama, una de las patologías que con más frecuencia se trata en los Servicios de Radioterapia, pero son aplicables en general a cualquier otra patología.

En el tratamiento del cáncer de mama la radioterapia se emplea como tratamiento adyuvante a la cirugía. El tumor primario se extirpa y posteriormente se administra una dosis de radiación sobre el lecho tumoral y sobre el resto de la mama.

Definiendo un volumen de tratamiento que comprenda el lecho tumoral más un margen para incluir la posible enfermedad microscópica, la experiencia clínica apunta a que se tiene una alta probabilidad de éxito en el tratamiento si la dosis que se administra a ese volumen es de 60 Gy en 30 fracciones. Sobre el volumen que cubre toda la mama la dosis administrada debe ser de 50 Gy en 25 fracciones.

Los efectos adversos del tratamiento se cuantifican fundamentalmente por la dosis que reciba el pulmón ipsilateral, siendo un buen indicador predictivo el porcentaje de volumen que recibe una dosis de 20 Gy o superior en relación al volumen total de ese pulmón. Se considera tolerable un tratamiento en el que el volumen de pulmón que recibe 20 Gy es inferior al 25\% del volumen total. Otro efecto adverso es la aparición de radiodermitis por respuesta aguda de la piel a la radiación. Este efecto es controlado directamente por el oncólogo de radioterapia mediante revisiones en consulta durante el curso del tratamiento. En casos graves puede complicar el desarrollo del tratamiento obligando a parar y descansar durante un tiempo.

La manera convencional de aplicar el tratamiento es hacerlo de manera secuencial. Durante 25 fracciones se irradia la mama completa para generar una distribución de dosis todo lo uniforme posible procurando evitar el pulmón. El valor medio de esa distribución de dosis será de 50 Gy y la dispersión de la dosis debe ser tal que no más de un 5\% de la mama reciba más de 53.5 Gy ni más de un 5\% de la mama reciba menos de un 47.5 Gy, o dicho en términos relativos, el 95\% del volumen de la mama recibe al menos el 95\% de la dosis de prescripción y no más del 5\% del volumen de la mama recibe más del 107\% de la dosis de prescripción. A continuación se administraran otras cinco fracciones en las que se procurará irradiar únicamente el volumen del lecho tumoral hasta que en este volumen se tenga una distribución de dosis con un valor medio de 60 Gy y de nuevo con una dispersión tal que se cumplan los criterios del 5\% aplicados al 95\% y al 107\% de la dosis de prescripción. En el argot de la radioterapia a la irradiación del volumen reducido se la denomina *Boost* o Sobreimpresión.

Los avances técnicos han permitido realizar el tratamiento de manera concurrente. Los dos volúmenes se irradian en el mismo número de sesiones prescribiendo diferente dosis por sesión en cada uno de ellos. El tratamiento se acorta en el tiempo y *la distribución de dosis se optimiza en mayor medida*. La tendencia actual es realizar los tratamientos en menos fracciones. Los médicos suelen utilizar la palabra *concomitante* para referirse a este esquema de tratamiento.

En radioterapia la evaluación de los planes de tratamiento se realiza mediante mapas de dosis sobre cortes tomográficos e histogramas dosis volumen (DVH). Los oncólogos de radioterapia valoran el cubrimiento de los volúmenes de tratamiento con las dosis de prescripción, pero también la conformación de la dosis a los volúmenes de tratamiento y la protección de las estructuras de riesgo. Los DVH reducen la información de la distribución de dosis permitiendo evaluar la cantidad de volumen irradiado con una dosis dada. Lo habitual es presentarlos como histogramas acumulados invertidos de manera que cada punto del histograma nos da el volumen que recibe al menos la dosis considerada.

## Módulo `platipy`

Para realizar las actividades propuestas en este cuaderno emplearemos el módulo [`platipy`](https://pyplati.github.io/platipy/getting_started.html).

Mediante este módulo se pueden generar gráficos que permiten visualizar los volúmenes de tratamiento y evaluar las distribuciones de dosis. Los sistemas comerciales de planificación en radioterapia ofrecen estas mismas visualizaciones. Sin embargo, las condiciones de los cuadernos **jupyter** limitan cómo se puede interaccionar con estos gráficos, siendo por tanto menos versátiles, si bien esencialmente tienen la misma funcionalidad. Alternativamente, para acercarse más a la experiencia que un software comercial proporciona, se puede emplear [**openTPS**](https://opentps.org/) para visualizar las imágenes y estructuras de planificación e interaccionar con las distribuciones de dosis.

## Visualización de distribuciones de dosis

Esta primera parte del cuaderno visualizaremos el conjunto de datos típico de un tratamiento de radioterapia: escáner CT, volúmenes de tratamiento y órganos de interés, distribuciones de dosis según diferentes esquemas de tratamiento.

Comenzamos por instalar el módulo `platipy`, que no está instalado por defecto en **Colab**, e importar una serie de módulos que nos harán falta

In [ ]:
!uv pip install platipy
import os
import SimpleITK as sitk

La siguiente celda descarga de la nube los datos que vamos a emplear.

Los datos se almacenan en la carpeta `/content/data` dentro de nuestra máquina **Colab**. Son por tanto volátiles, los datos se borrarán cuando el entorno de ejecución se desconecte. Téngase en cuenta también que la carpeta `/content` es a la que apunta por defecto el explorador de archivos en **Colab**, la carpeta `data` que aparecerá en nuestro explorador será con la que trabajaremos.

Los datos incluyen las imágenes de un CT de simulación, un archivo de estructuras con los contornos de los volúmenes de tratamiento y de los órganos de interés, y varias distribuciones de dosis correspondientes a diferentes esquemas de tratamiento. También hay datos de planes de tratamiento, pero no son relevantes para este cuaderno. Están en formato **DICOM**. La estructura de carpetas sigue el orden convenido por el paquete [`scikit-rt`](https://scikit-rt.github.io/scikit-rt/patients_and_studies.html), y es autodescriptiva.



In [ ]:
%%bash

# Crear el directorio de datos
rm -rf /content/data
mkdir -p /content/data

# Ejecutar código Python para descargar los datos del Drive
python << EOF > /dev/null 2>&1
# Descargar los datos compartidos de Google Drive
import gdown
url = 'https://drive.google.com/file/d/1ICECuwHhMBoMwioQ96-UpkV43v0kNneI/view?usp=sharing'
output = '/content/data/MamaDer001.zip'
gdown.download(url, output, quiet=True, fuzzy=True)
EOF

# Preparación de los datos para realziar el cuaderno de Radiobiología

# Descomprimir archivo de datos
unzip /content/data/MamaDer001.zip -d /content/data >/dev/null

# Eliminar el archivo comprimido
rm -f /content/data/MamaDer001.zip

`platipy` maneja las imágenes como objetos [imagen `SimpleITK`](https://simpleitk.org/doxygen/latest/html/classitk_1_1simple_1_1Image.html)

La siguiente celda lee el estudio CT de simulación y crea el objeto imagen

In [ ]:
reader = sitk.ImageSeriesReader()

dicom_names = reader.GetGDCMSeriesFileNames('/content/data/MamaDer001/20230116_095704/CT/20230116_095704')
reader.SetFileNames(dicom_names)

image = reader.Execute()

El objeto `image` expone una colección de métodos para trabajar con la imagen Si queremos podemos conocer el número de pixeles y su tamaño por ejemplo

In [ ]:
print(f"El tamaño de la imagen es {image.GetSize()} y los tamaños del voxel en mm son: {image.GetSpacing()}")

En formato **DICOM** las estructuras se guardan en un archivo único de forma natural como secuencias de puntos que definen los contornos de las mismas.

`platipy`las maneja como máscaras de imagen en formato **nifti** y tiene una función para realizar la conversión. Una máscara es una imagen binaria, con todos sus vóxeles a cero salvo los que se encuentran dentro del contorno de la estructura, que tienen valor uno.

El conjunto de todas las estructuras se tiene que almacenar en un diccionario python con el nombre de la estructura como clave y la imagen como valor.

La siguiente celda hace la conversión de **DICOM** a **nifti**, guarda las máscaras en una carpeta dentro de `/content/data` y crea el diccionario de estructuras

In [ ]:
!rm -rf /content/data/nifti/volumes/
!mkdir -p /content/data/nifti/volumes/
output_dir = '/content/data/nifti/volumes/'
from platipy.dicom.io.rtstruct_to_nifti import convert_rtstruct
convert_rtstruct(dcm_img='/content/data/MamaDer001/20230116_095704/CT/20230116_095704',
                 dcm_rt_file='/content/data/MamaDer001/20230116_095704/RTSTRUCT/CT/20230116_095704/RS.1.2.246.352.205.5566667512400723974.375329800850442140.dcm',
                 output_dir=output_dir,
                 prefix="")
volumefiles = [f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))]
contours = {}
for volumefile in volumefiles:
    _name = volumefile.split(".nii.gz")[0]
    contours[_name] = sitk.ReadImage(os.path.join(output_dir, volumefile))

Para visualizar las imágenes `platipy` ofrece el objeto `ImageVisualizer`

Al constructor del visualizador se le pasa la imagen y los parámetros de corte. Muestra tres panales con los cortes en los planos axial, coronal y sagital.

Se pueden superponer los contornos pasándole el diccionario de contornos.

### Tarea
----
Correr la siguiente celda y jugar con los parámetros de corte `cut` para entender su función.


In [ ]:
image.GetSpacing()

In [ ]:
from platipy.imaging import ImageVisualiser
image_visualiser = ImageVisualiser(image, cut=(35,256,256))
image_visualiser.add_contour(contours)
fig = image_visualiser.show()

Hay varias acciones que se pueden hacer para agilizar y mejorar la representación anterior.

Lo primero es representar solo las estructuras que son relevantes. Para este cuaderno, y en general en el tratamiento de la mama derecha, las estructuras de interés son los dos PTVs y el pulmón derecho. Creamos un diccionario de contornos que solo contenga estas estructuras.

Lo segundo es utilzar una estructura como referencia para centrar la región visualizada. La función `set_limits_from_label` del objeto `ImageVisualiser` sirve para esto. A la función se le puede pasar también el parámetro `expansion` para dar un margen alrededor de la estructura.

### Tarea
----
Particularizar la siguiente celda para que solo se representen `PTV_mama_dcha, PTV_boost_dcho, Pulmon_Dcho` y la representación se centre en el `PTV_boost_dcho` con un margen suficiente para que se vea `PTV_mama_dcha` completo.

In [ ]:
image_visualiser = ImageVisualiser(image, cut=(20, 200, 150))
contours_to_show_list = [] # Introducir en la lista los nombres de las estructuras a visualizar
contour_limit = '' # Particularizar con el nombre la estructura en la que queremos que se centre la representación
expansion = 0 # Introducir un valor de expansión suficiente para que se vea completo el PTV_mama_dcha
contours_to_show = {key: contours[key] for key in contours_to_show_list if key in contours}
image_visualiser.add_contour(contours_to_show)
image_visualiser.set_limits_from_label(contours[contour_limit], expansion=expansion)
fig = image_visualiser.show()

> Una mejor manera de centrar las imágenes es utilizar el centro de masas de una estructura. Ver la documetación de la función [`get_com`](https://pyplati.github.io/platipy/utils.html).

Para representar la dosis es necesario recubrir (*overlay*) con ella la imagen de CT. `sitk.ReadImage` puede crear un mapa de dosis como una imagen escalar a partir de un archivo **DICOM** RT Dose. Para recubrir una imagen con otra es necesario que las dimensiones y la frecuencia espacial de muestreo en ambas imágenes coincidan. La función `sitk.resample(image, image, ...)` se encarga de modificar los parámetros de una imagen según los de la otra.

En `platipy` el objeto [`ImageVisualiser`](https://pyplati.github.io/platipy/visualiser.html) ofrece la función [`add_scalar_overlay`](https://pyplati.github.io/platipy/visualiser.html) para realizar el recubrimiento. Esta función incluye el parámetro `colormap` para hacer más gráfico el mapa de dosis. Es una decisión arbitraria el mapa que se elija. Si por ejemplo se usa el mapa de colores `jet` las dosis mayores se representan en tonos calientes y los colores tienden a tonos más fríos a medida que la dosis desciende.

En la siguiente celda se importan los módulos necesarios para obtener el centro de masas de una imagen y representar la dosis.

In [ ]:
from platipy.imaging.label.utils import get_com
import pydicom as dcm
import matplotlib

La siguiente celda sirve para mostrar el esquema de código para hacer una sobreimposición de la dosis

In [ ]:
dose_file_path = '/content/data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.47352446189117142439988284232660965553.dcm'
dose = sitk.ReadImage(dose_file_path)
dose = sitk.Resample(dose, image)

vis = ImageVisualiser(image, cut=get_com(contours["PTV_boost_dcho"]))

vis.add_scalar_overlay(dose, discrete_levels=20, colormap=matplotlib.colormaps.get_cmap("jet"), name="Dose (Gy)")
vis.add_contour(contours_to_show)

fig = vis.show()

Si nos fijamos veremos que los valores de la dosis son absurdos. Si revisamos la documentación del estándar **DICOM** veremos que dice que los valores se guardan en formato de entero largo renormalizados para ajustar su rango. El valor de escala se guarda en el propio archivo de dosis bajo una etiqueta denominada `DoseGridScaling`.

### Tarea
-----

Escribir una modificación de la celda anterior que:
  - Lea el archivo DICOM mediante el moóulo `pydicom` en un DataSet y acceda al valor `DoseGridScaling`
  ```
      dose_dataset = dcm.dcmRead(dose_file_path)
  ```
  -  multiplique el valor de la dosis, variable `dose`, por `dose_dataset.DoseGridScaling`. Para que la multiplicación tenga sentido es necesario también hacer una conversión del tipo de datos de la imagen, de entero largo a Float o Double. Emplear para ello la función `sitk.Cast()`
  ```
      dose = sitk.Cast(dose, sitk.sitkFloat32)
  ```

### Tarea
----

Ejecutar la siguiente celda para testear que nuestros resultados son correctos

Debería verificarse:

$60 \,\text{Gy} \cdot 1.07 <= D_\text{max} [\text{Gy}] < 60\,\text{Gy}  \cdot 1.1$

In [ ]:
import numpy as np

# Get the maximum value of the dose image
max_dose = np.max(sitk.GetArrayFromImage(dose))

# Define the expected range
lower_bound = 60 * 1.02
upper_bound = 60 * 1.15

# Check if the maximum dose is within the range
assert max_dose >= lower_bound and max_dose < upper_bound, f"Maximum dose ({max_dose:.2f} Gy) is not within the expected range [{lower_bound:.2f} Gy, {upper_bound:.2f} Gy)"

print(f"Test pasado: Dosis máxima ({max_dose:.2f} Gy) dentro del rango esperado [{lower_bound:.2f} Gy, {upper_bound:.2f} Gy)")

**No** continuar con el resto del cuaderno si el mensaje de `Test pasado` no aparece al correr la celda anterior.

### Tarea
----

Duplicar la celda anterior para visualizar la distribución de dosis contendida en:

`dose_file_path = '/content/data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.52708297311437444114737711031232263301.dcm'
`

### Cuestión
-----

1. ¿Cuál es el valor de la dosis máxima prevista para el esquema de tratamiento representado en la celda anterior?

### Respuesta
----
**Tu respuesta aquí**

### Cuestión
----
2. Desplazándose por las imágenes de CT determinar cuál es la extensión en la dirección craneo caudal, (indicando el corte inferior y el corte superior en los que se visualiza estructura) para las estructuras
- PTV mama dcha
- PTV boost dcho
- Pulmón dcho

> El corte inferior es el corte en el extremo de la dirección caudal y el corte superior el corte en el extremo de la dirección craneal

#### Respuesta
----
**Tu respuesta aquí**

### Cuestión
----

3. En las imágenes se pueden apreciar unos objetos brillantes colocados sobre la superficie de la paciente. Sirven para señalar la cicatriz resultado de la intervención quirúrgica previa a la radioterapia que se realizó para extirpar el tumor. Determinar la extensión de la cicatriz en la dirección craneo caudal.

### Respuesta
----
**Tu respuesta aquí**

### Cuestión
-----

4. Indicar razones que justifiquen la extensión diferente de la cicatriz y del volumen PTV_boost_dcho.

### Repuesta
----
**Tu respuesta aquí**


### Cuestión
----

5. Dentro de la mama se pueden apreciar puntos brillantes que claramente no son de origen natural, se trata de grapas quirúrgicas. Indicar en qué cortes tomográgicos se ven grapas.

### Respuesta
----
**Tu respuesta aquí**


### Cuestión
----
6. Indicar si todas las grapas se encuentran dentro del volumen PTV_boost_dcho y comentar la relación entre la posición de las grapas y la del PTV boost.

### Respuesta
----
**Tu respuesta aquí**


El parámetro `min_value` del método `add_scalar_overlay` del objeto `ImageVisualizer` se puede utilizar para fijar el valor de dosis a partir del que se representa la distribución de dosis. De esta manera podemos realizar una comprobación visual del grado de cobertrura de los volúmenes de tratamiento

### Tarea
----
Realizar representaciones de la distribuciones de dosis correspondientes a las dos fases del tratamiento secuencial (mama y boost) y al tratamiento suma, en las que se visualice el grado de cobertura de la dosis de tratamiento del volumen correspondiente.
> Particularizar  `dose_file_path` con el archivo correspondiente a cada fase y a la suma

> Para fijar 'min_value` téngase en cuenta que los volúmenes se consideran tratados a partir del 95% de la dosis de prescripción

### Cuestión
----
7. A partir de las representaciones de la distribuciones de dosis, realizar una evaluación cualitativa del grado de cubrimiento de los volúmenes de tratamiento.

### Respuesta
----
**Tu respuesta aquí**

### Tarea
----
*Jugando* con el parámetro `min_value` en la representación de la distribución de dosis correspondiente al tratamiento concurrente, hacer una estimación de las dosis de prescripción (mama y boost)

### Cuestión
----
8. Indicar las dosis de prescripción estimadas para el tratamiento concurrente

### Respuesta
----

| Volumen | Dosis de prescripción estimada [Gy] |
|----|-----|
|PTV_mama_dcha |  |
|PTV_boost_dcho |  |



## Histogramas dosis volumen

Como ya hemos indicado en **Fundamentos** visualizar la dosis permite valorar su distribución espacial pero no es un método sencillo de realizar evaluaciones cuantitativas.

En esta parte del cuaderno trabajaremos con gráficas de *DVH*, histograma dosis volumen, para ayudarnos a comprender en qué consisten estos objetos.

`platpy` contiene herramientas con las funcionalidades necesarias para el cálculo de DVHs y para la evaluación de tratamientos mediante su inspección.

En la siguiente celda importamos las funciones relevantes para esta parte y el módulo `matplotlib.pyplot` para generar gráficos DVH.


In [ ]:
from platipy.imaging.dose.dvh import calculate_dvh_for_labels, calculate_d_x, calculate_v_x
import matplotlib.pyplot as plt

El DVH nos da información estadística sobre la cantidad de dosis recibida por un volumen dado de un órgano. La función [`calculate_dvh_for_labels(dose, labels)`](https://pyplati.github.io/platipy/dose.html) nos calcula dada una distribución de dosis `dose` el histograma para cada estructura en el diccionario `labels`

La función devuelve el histograma como un `DataFrame pandas` en el que el índice es cada uno de los nombres de las estructuras y las columnas son: el volumen total de la estructura en cc, la dosis media, y las fracciones de volumen del órgano cubiertas por la dosis dada en el encabezado de la columna.

### Tarea
----
Ejecutar la siguiente celda para ver el contenido de esta estructura de datos

In [ ]:
dvh = calculate_dvh_for_labels(dose, contours_to_show)
dvh

Al ser un `DataFrame pandas`, `dvh` tiene sus propios métodos para su representación gráfica. Cómo queremos un gráfico por cada estructura necesitamos trasponerlo primero. Mediante `matplotlib.pyplot` podemos dar formato al gráfico.

La siguiente celda representa `dvh` del modo en el que normalmente se muestra en los planificadores, con el volumen expresado en tanto por ciento del volumen total.

In [ ]:
dose_file_path = '/content/data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.47352446189117142439988284232660965553.dcm'
dose = sitk.ReadImage(dose_file_path)
dose = sitk.Cast(dose, sitk.sitkFloat32)
dose = sitk.Resample(dose, image)
dose_dataset = dcm.dcmread(dose_file_path)
dose = dose * dose_dataset.DoseGridScaling

dvh = calculate_dvh_for_labels(dose, contours_to_show)
# Reshape the DVH
dvh = dvh.set_index("label")
dvh = dvh.rename_axis("Dose")
dvh = dvh.iloc[:,3:].transpose()
dvh = dvh * 100

# Plot the DVH
fig, ax = plt.subplots()
dvh.plot(ax=ax, kind="line", colormap=matplotlib.colormaps.get_cmap("rainbow"), legend=False)

# Add labels and show plot
plt.legend(loc='best')
plt.xlabel("Dose (Gy)")
plt.ylabel("Volume [%]")
plt.title("Dose Volume Histogram (DVH)")
plt.show()

Consideremos la dosis de los dos esquemas de tratamiento. El sufijo `_sib` se refiere al esquema *Boost integrado* (***S**imultaneous **I**ntegrated **B**oost*) y el sufijo `_seq` a *Secuencial* (***Seq**uential*) o concomitante.
En la siguiente celda creamos los objetos que la contendrán.

In [ ]:
dose_seq_file_path = '/content/data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.47352446189117142439988284232660965553.dcm'
dose_seq = sitk.ReadImage(dose_seq_file_path)
dose_seq = sitk.Cast(dose_seq, sitk.sitkFloat32)
dose_seq = sitk.Resample(dose_seq, image)
dose_seq_dataset = dcm.dcmread(dose_seq_file_path)
dose_seq = dose_seq * dose_seq_dataset.DoseGridScaling

dose_sib_file_path = '/content/data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.52708297311437444114737711031232263301.dcm'
dose_sib = sitk.ReadImage(dose_sib_file_path)
dose_sib = sitk.Cast(dose_sib, sitk.sitkFloat32)
dose_sib = sitk.Resample(dose_sib, image)
dose_sib_dataset = dcm.dcmread(dose_sib_file_path)
dose_sib = dose_sib * dose_sib_dataset.DoseGridScaling

Dado que un *DVH* es una curva que relaciona dosis con volumen, los objetivos de prescripción y las restricciones de dosis que se imponen sobre los *DVH* pueden venir expresadas de dos maneras diferentes:

- **Dxx** nos dice la dosis que recibe un volumen **xx**. Si **xx** se expresa sin unidades se refiere a un porcentaje de volumen respecto al volumen total del órgano. Si se da con unidades se refiere lógicamente a volumen medido en esas unidades. Las propias dosis se pueden dar en valor absoluto medido en las unidades de dosis correspondiente o en valores relativos respecto a la dosis de prescripción del tratamiento.
- **Vxx** nos dice el volumen de órgano que recibe la dosis **xx**. En este caso lo más habitual es que si no aparecen unidades se refiera a un valor de dosis en Gy. Por ejemplo un V20 inferior al 25% indicaría que se intenta conseguir que la dosis de 25 Gy sea recibida por menos del 25% del volumen del órgano afectado.

Las siguientes celdas muestran ejemplos de cómo *interrogar* a los objetos `dvh` sobre estos parámetros

In [ ]:
# Cálculo de D20 del pulmón derecho (dosis que recibe el 20% del pulmón derecho)

dvh_seq = calculate_dvh_for_labels(dose_seq, contours_to_show)
calculate_d_x(dvh=dvh_seq, x=20, label='Pulmon_Dcho')

In [ ]:
# Cálculo de V20 del pulmón derecho (Volumen de pulmón derecho que recibe 20 Gy o más)

dvh_seq = calculate_dvh_for_labels(dose_seq, contours_to_show)
calculate_v_x (dvh=dvh_seq, x=20, label='Pulmon_Dcho')

Si el parámetro `label` no se especifica, se calcula el parámetro para todas las estructuras de `dvh`

In [ ]:
calculate_v_x (dvh=dvh_seq, x=55)

En el resultado anterior vemos que `calculate_v_x` devuelve el volumen `V` en centímetros cúbicos. Suele ser habitual que se exprese en tanto por ciento del volumen total del órgano.

La siguiente función se encarga de añadir al `DataFrame` de resultados una columna con `V` en porcentaje.

In [ ]:
def calculate_d_x_perc(dvh, x, label=None):
  vdf = calculate_v_x(dvh=dvh, x=x, label=label)
  # Rename the V20 column to Vx%
  if label is None:
    vdf = vdf.merge(dvh[['label', 'cc']], on='label')
    cols = ['label', 'cc', f'V{x}']
    vdf = vdf[cols]
    vdf[f'V{x}%'] = vdf[f'V{x}'] / vdf.cc * 100
  else:
    vdf[f'V{x}%'] = vdf[f'V{x}'] / dvh[dvh.label == label].cc.values * 100
  return vdf

In [ ]:
dvh_seq = calculate_dvh_for_labels(dose_seq, contours_to_show)
calculate_d_x_perc(dvh = dvh_seq, x=20, label='Pulmon_Dcho')

De nuevo, si `label` no se especifica la función devuelve el valor para todas las estructuras presentes en `dvh`

In [ ]:
calculate_d_x_perc(dvh = dvh_seq, x=20)

### Cuestión
-----
9. Realizar una evaluación de los siguientes criterios de prescripción de tratamiento y de tolerabilidad del tratamiento sobre el plan correspondiente. Indicar Sí en la columna **Cumplido**  si el criterio se *cumple* o No si *no se cumple*


### Respuesta
-----
| Plan | Volumen | Criterio | Cumplido |
|----|-----|------|---|
|Mama | PTV_mama_dcha | Al menos 95% cubierto por el 95% de la dosis de prescripción (50 Gy) | |
| | PTV_mama_dcha | No más del 5% cubierto por el 107% de la dosis de prescripción (50 Gy) | |
| | PTV_boost_dcho | Al menos 95% cubierto por el 95% de la dosis de prescripción (50 Gy) | |
| | PTV_boost_dcho | No más del 5% cubierto por el 107% de la dosis de prescripción (50 Gy) | |
| | Pulmon_Dcho | Volumen de órgano que recibe 20 Gy o más inferior al 25% | |
|Boost | PTV_boost_dcho | Al menos 95% cubierto por el 95% de la dosis de prescripción (10 Gy) | |
| | PTV_boost_dcho | No más del 5% cubierto por el 107% de la dosis de prescripción (10 Gy) | |
|Suma | PTV_boost_dcho | Al menos 95% cubierto por el 95% de la dosis de prescripción (60 Gy) | |
| | PTV_boost_dcho | No más del 5% cubierto por el 107% de la dosis de prescripción (60 Gy) | |
| | Pulmon_Dcho | Volumen de órgano que recibe 20 Gy o más inferior al 25% | |
|SIB | PTV_mama_dcha | Al menos 95% cubierto por el 95% de la dosis de prescripción (40 Gy) | |
| | PTV_mama_dcha | No más del 5% cubierto por el 107% de la dosis de prescripción (40 Gy) | |
| | PTV_boost_dcho | Al menos 95% cubierto por el 95% de la dosis de prescripción (48 Gy) | |
| | PTV_boost_dcho | No más del 5% cubierto por el 107% de la dosis de prescripción (48 Gy) | |
| | Pulmon_Dcho | Volumen de órgano que recibe 18 Gy o más inferior al 25% | |


In [ ]:
# Introducir las celdas de código que sean necesarias para realizar la evaluación del cumplimiento de criterios

# Radiobiología
## Fraccionamiento y equivalencia de tratamientos

En esta parte del cuaderno queremos utilizar el modelo lineal cuadrático para justificar los valores de prescripción y de restricción sobre el pulmón que hemos utilizado en el apartado anterior.

Según el modelo lineal cuadrático una dosis absorbida $D$ administrada en un número de fracciones $n$ produce una dosis biológica equivalente $DBE$ dada por
\begin{equation}
DBE = D\,(1 + \frac{D/n}{\alpha/\beta})
\end{equation}

donde el cociente $\alpha/\beta$ [Gy] es un parámetro específico del tejido que recibe la dosis. En el resto del cuaderno consideraremos un $\alpha/\beta = 3$ tanto para el tejido de la mama como para el pulmón.

Dos esquemas de tratamiento son equivalentes si producen la misma $DBE$.

### Cuestión
----

10. Calcular qué dosis se tiene que administrar al volumen PTV boost dcho si se quiere realizar el tratamiento en lugar de en forma secuencial, 25+5 fracciones, en forma concurrente, 25 fracciones.

### Respuesta
---
**Tu respuesta aquí**




### Cuestión
----
11. Repetir el cálculo anterior si queremos acelerar el tratamiento a 15 fracciones.

### Respuesta
----
**Tu respuesta aquí**


### Cuestión
----

12. Calcular para el tratamiento concurrente en 15 fracciones cuál es la nueva restricción que se tiene que imponer sobre el pulmón para tener la misma probablidad de complicaciones que aceptamos en el tratamiento secuencial.

### Respuesta
----
**Tu respuesta aquí**

## Comparación de planes mediante DVHs


Para realizar la comparación podemos combinar los DVHs de diferentes esquemas de tratamiento en una misma figura.  Personalmente prefiero además que la dosis no sea el índice del DataFrame. El siguiente código reconfigura los DataFrames correspondientes al tratamiento secuencial y al concomitante.

In [ ]:
## Tratatamiento secuencial
dvh_seq = calculate_dvh_for_labels(dose_seq, contours_to_show)
# Reshape the DVH
dvh_seq = dvh_seq.set_index("label")
# Transpose the DVH to have dose values as the index
dvh_seq = dvh_seq.iloc[:,3:].transpose()
dvh_seq = dvh_seq * 100

# Convert the index (which holds dose values) into a column named 'D'
dvh_seq = dvh_seq.reset_index(names=['D'])

# Calculate the 'Dr' column using the new 'D' column
dvh_seq['Dr'] = dvh_seq['D']/60. * 100

# Get the list of all columns
cols = dvh_seq.columns.tolist()

# Define the desired order: 'D', 'Dr', and then the rest of the columns (structures)
other_columns = [col for col in cols if col not in ['D', 'Dr']]
desired_order = ['D', 'Dr'] + other_columns

# Reorder the DataFrame columns
dvh_seq = dvh_seq[desired_order]


## Tratamiento concomitante
dvh_sib = calculate_dvh_for_labels(dose_sib, contours_to_show)
# Reshape the DVH
dvh_sib = dvh_sib.set_index("label")
# Transpose the DVH to have dose values as the index
dvh_sib = dvh_sib.iloc[:,3:].transpose()
dvh_sib = dvh_sib * 100

# Convert the index (which holds dose values) into a column named 'D'
dvh_sib = dvh_sib.reset_index(names=['D'])

# Calculate the 'Dr' column using the new 'D' column
dvh_sib['Dr'] = dvh_sib['D']/60. * 100

# Get the list of all columns
cols_sib = dvh_sib.columns.tolist()

# Define the desired order: 'D', 'Dr', and then the rest of the columns (structures)
other_columns_sib = [col for col in cols_sib if col not in ['D', 'Dr']]
desired_order_sib = ['D', 'Dr'] + other_columns_sib

# Reorder the DataFrame columns
dvh_sib = dvh_sib[desired_order_sib]

# Representar gráficamente
fig, ax = plt.subplots()

dvh_seq.plot.line(x='D', y='PTV_boost_dcho', color='r', ax=ax, label='Seq PTV_boost_dcho')
dvh_seq.plot.line(x='D', y='PTV_mama_dcha', color='b', ax=ax, label ='Seq PTV_mama_dcha')
dvh_seq.plot.line(x='D', y='Pulmon_Dcho', color='g', ax=ax, label='Seq Pulmon_Dcho')

dvh_sib.plot.line(x='D', y='PTV_boost_dcho', color='m', ax=ax, linestyle='-', label='SIB PTV_boost_dcho')
dvh_sib.plot.line(x='D', y='PTV_mama_dcha', color='y', ax=ax, label='SIB PTV_mama_dcha')
dvh_sib.plot.line(x='D', y='Pulmon_Dcho', color='b', ax=ax, label='SIB Pulmon_Dcho')

plt.legend(bbox_to_anchor=(1., 0.9))

ax.set_ylabel('% Volumen')
ax.set_xlabel('D [Gy]')

plt.show()

### Tarea
----
En la siguiente celda programar código que añada a cada DataFrame sendas columnas con la **DBE** dosis biológica equivalente y la **DBEQ2**  correspondientes a la dosis **D**.

>Téngase en cuenta el fraccionamiento (la dosis por fracción) utilizada en cada esquema de tratamiento.



Definimos la Dosis Equivalente Biológica como una función de la Dosis, el número de fracciones y el coeficinte $\alpha/\beta$

In [ ]:
def DBE_f(D, n, α_β=3):
    '''
    Calcula la Dosis Biológica Equivalente
    Argumentos:
    D: Dosis absorbida [Gy]
    n: Número de fracciones
    α_β: Razón α/β del volumen considerado [Gy]
    '''

def DBEQ2_f(D, n, α_β=3):
    '''
    Calcula la Dosis Biológica Equivalente en fraccines de 2 Gy
    Argumentos:
    D: Dosis absorbida [Gy]
    n: Número de fracciones
    α_β: Razón α/β del volumen considerado [Gy]
    '''


Añadimos a cada DataFrame las columnas de dosis equivalente

In [ ]:
dvh_seq['DBE'] = DBE_f(dvh_seq.D, 30)
dvh_sib['DBE'] = DBE_f(dvh_sib.D, 15)
dvh_seq['DBEQ2'] = DBEQ2_f(dvh_seq.D, 30)
dvh_sib['DBEQ2'] = DBEQ2_f(dvh_sib.D, 15)

### Tarea
----
En la siguiente celda programar código que genere una figura que represente de forma conjunta los DVH del tratamiento secuencial y el concomitante en función  de la dosis biológica equivalente


### Cuestión
------
13. Analizar la equivalencia de los tratamientos expresados en **DBE**.
> **Nota**: Dos tratamientos son equivalentes si sus DVH son similares. Esto quiere decir que teniendo presente su aplicación a la práctica clínica, los PTVs tienen un valor medio, cobertura, uniformidad y valor máximo  aproximadamente iguales. La forma de la curva de los órganos de riesgo tiene un aspecto similar, en especial los valores relacionados con las restriccciones asociadas a su tolerabilidad son parecidos.

### Respuesta
-----
**Tu respuesta aquí**

### Cuestión
------
14. ¿Qué esquema de tratamiento produce una mejor conformación? Razonar la respuesta comentando las diferencias en los DVHs de los tres volúmenes.
> **Conformación** de la dosis es el grado de coincidencia espacial entre la distribución de la dosis y los volúmenes de tratamiento

### Respuesta
-----
**Tu respuesta aquí**


### Cuestión
------
15. Identificar qué PTV tiene una mayor grado de acuerdo en términos de **DBE** entre ambos esquemas de tratamiento y explicar qué puede estar produciendo esta diferencia dosimétrica en el otro PTV.

### Respuesta
-----
**Tu respuesta aquí**

### Tarea
----
En la siguiente celda programar código que genere una figura que represente de forma conjunta los DVH del tratamiento secuencial y el concomitante en función de **DBEQ2**

### Cuestión
------
16. Razonar si existen diferencias en la equivalencia de los tratamientos si se analizan en términos de **DBE** o de **DBEQ2**.

### Respuesta
-----
**Tu respuesta aquí**

### Tarea
----
En la siguiente celda programar código que genere una figura que represente de forma conjunta los DVH del tratamiento concomitante a 2 Gy por fracción y del secuencial en términos de **DBEQ2**

### Cuestión
------
17. Explicar por qué los DVHs no coinciden a pesar de que el concepto **DBEQ2** se introdujo para que los Oncólogos de Radioterapia pudieran comparar tratamientos realizados en fraccionamiento estándar de 2 Gy, que es cómo se ha realizado el tratamiento Suma.

### Respuesta
-----
**Tu respuesta aquí**